In [22]:
import os
import time
import joblib
from pathlib import Path
import numpy as np

import processing
import constants
import helpers

def get_image_descriptors(image):
    processed = processing.filter_image(image)
    moments = processing.calculate_image_moments(processed)

    moments = np.array([moment[0] for moment in moments], dtype=np.float32)
    moments = np.abs(moments)
    moments = np.log10(moments)

    return moments

def describe_image(image, label):
    descriptors = get_image_descriptors(image)
    return (np.array(descriptors), label)

def describe_images_in_class(images, label):
    classDescriptors = joblib.Parallel(n_jobs=5)(joblib.delayed(describe_image)(image, label) for image in images)
    classDescriptors = np.array(classDescriptors, dtype=object)

    print('class', classDescriptors.shape)

    return classDescriptors

def build_image_descriptors(imagesByLabels):
    labeledDescriptors = np.empty((0,2))

    for label in range(len(imagesByLabels)):
        labeledDescriptors = np.concatenate((labeledDescriptors, describe_images_in_class(imagesByLabels[label], label)))

    print('labeled1', labeledDescriptors.shape)

    return labeledDescriptors

def build_set(set_type):
    start_time = time.perf_counter()

    print(f'Loading {set_type} dataset...')
    labels = np.array([0,1,2,3,4])
    imagesByLabels = [helpers.get_and_enrich_all_images_from_directory(Path(constants.DATASET_DIR, set_type, str(classType))) for classType in labels]
    print(f'Loaded {sum(len(l) for l in imagesByLabels)} images (with extensions) in {time.perf_counter() - start_time} s')

    print(f'Getting image descriptors...')
    labeledDescriptors = build_image_descriptors(imagesByLabels)
    print(f'Total image loading time: {time.perf_counter() - start_time} s...')

    descriptors, labels = zip(*[(item[0], item[1]) for item in labeledDescriptors])
    descriptors = np.array(descriptors)
    labels = np.array(labels)

    # print(descriptors[:4], labels)

    return labels, descriptors

In [3]:
# Gets the histogram sets with labels
print('Loading datasets...')
load_time_start = time.perf_counter()

labels = np.array([0,1,2,3,4])
y_train, x_train = build_set('train')
y_test, x_test = build_set('val')

print(f'Loaded datasets. Time taken: {time.perf_counter() - load_time_start} s')

Loading datasets...
Loading train dataset...
Loaded 17334 images (with extensions) in 9.352803600000001 s
Getting image descriptors...
class (6858, 2)
class (3138, 2)
class (4548, 2)
class (2271, 2)
class (519, 2)
labeled1 (17334, 2)
Total image loading time: 92.60298 s...
Loading val dataset...
Loaded 2478 images (with extensions) in 1.3310684000000066 s
Getting image descriptors...
class (984, 2)
class (459, 2)
class (636, 2)
class (318, 2)
class (81, 2)
labeled1 (2478, 2)
Total image loading time: 13.198375900000002 s...
Loaded datasets. Time taken: 105.9452404 s


In [13]:
from sklearn.preprocessing import RobustScaler, scale

print(x_train[:2])

x_scaled = x_train
x_test_scaled = x_test

scaler = RobustScaler()

x_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

x_scaled = scale(x_train)
x_test_scaled = scale(x_test)

print(x_scaled[:2])

In [24]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=60, n_jobs=-1)
clf.fit(x_scaled, y_train)

KNeighborsClassifier(n_jobs=-1, n_neighbors=60)

In [27]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = {
    'kernel': ( 'linear', 'rbf' ),
    'C': [0.1, 10, 100, 1000]
}

svc = SVC()
clf = GridSearchCV(svc, parameters, verbose=1, n_jobs=-1)
clf.fit(x_scaled, y_train)

clf = clf.best_estimator_

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [18]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge', verbose=0, n_iter_no_change=10000, penalty='elasticnet', alpha=1e-5, tol=1e-5, eta0=1e-4, learning_rate="adaptive", epsilon=1e-3, max_iter=70000)
clf.fit(x_scaled, y_train)

SGDClassifier(alpha=1e-05, epsilon=0.001, eta0=0.0001, learning_rate='adaptive',
              max_iter=70000, n_iter_no_change=10000, penalty='elasticnet',
              tol=1e-05)

In [28]:
from sklearn.metrics import confusion_matrix, classification_report

y_predict=clf.predict(x_test_scaled)
accuracy = clf.score(x_test_scaled, y_test)

print(f"Accuracy: {accuracy * 100}%")
print("\nClassification report:")
print(classification_report(y_test, y_predict))

print("\nConfusion matrix:")
print(f"Labels: {labels}\n")
print(confusion_matrix(y_test, y_predict, labels=labels))

Accuracy: 40.96045197740113%

Classification report:
              precision    recall  f1-score   support

           0       0.43      0.90      0.58       984
           1       0.04      0.00      0.00       459
           2       0.35      0.19      0.25       636
           3       0.18      0.02      0.04       318
           4       0.20      0.01      0.02        81

    accuracy                           0.41      2478
   macro avg       0.24      0.23      0.18      2478
weighted avg       0.30      0.41      0.30      2478


Confusion matrix:
Labels: [0 1 2 3 4]

[[882  10  82  10   0]
 [399   1  50   8   1]
 [491   8 124  10   3]
 [235   4  72   7   0]
 [ 53   0  24   3   1]]


In [ ]:
model_output_path = './model.xml'

if os.path.exists(model_output_path):
    joblib.dump(clf, model_output_path)
else:
    print("Cannot save trained svm model to {0}.".format(model_output_path))

Cannot save trained svm model to ./svc.model.
